# FLEE-GNN - Evaluation

## load libraries

In [3]:
import pandas as pd
import numpy as np

## resilience prediction with trained model weights

In [1]:
! python ./scripts/evaluation.py --info-dir ./data/info/ --data-path ./data/2012_data.csv --label-path ./data/2012_label.csv --model-path ./weights/test0707/fl_model_round_18.pth --output-dir ./training/fl_noise_output_02_0707/
! python ./scripts/plot_resilience.py --true-label-path ./data/2012_label.csv --predict-label-path ./training/fl_noise_output_02_0707/predict.csv --info-dir ./data/info/ --output-dir ./training/fl_noise_output_02_0707/

Namespace(data_path='./data/2012_data.csv', info_dir='./data/info/', label_path='./data/2012_label.csv', model_path='./weights/test0707/fl_model_round_18.pth', output_dir='./training/fl_noise_output_02_0707/')
Namespace(info_dir='./data/info/', output_dir='./training/fl_noise_output_02_0707/', predict_label_path='./training/fl_noise_output_02_0707/predict.csv', true_label_path='./data/2012_label.csv')


## load ground truth and predicted resilience

In [6]:
gt = pd.read_csv('./data/2012_label.csv')
gt_import_resilience = gt['import_resilience'].values

In [7]:
fl_pred = pd.read_csv('./training/fl_noise_output_02_0707/predict.csv', header=None)
fl_pred_import_resilience = fl_pred.values.flatten()

## evaluation metrics

In [10]:
# mean squared error
print("GT-PRED MSE: ", ((gt_import_resilience - fl_pred_import_resilience) ** 2).mean())

GT-PRED MSE:  0.011883658770327488


In [14]:
# top-k coincidence rate
def get_coincidence_rate_topk(gt, pred, top_k):
    gt1 = gt[(~np.isnan(gt)) & (~np.isnan(pred))]
    pred1 = pred[(~np.isnan(gt)) & (~np.isnan(pred))]
    total = gt1.shape[0]
    selected = top_k # int(np.ceil(top_pick * total))
#     print(f'selected samples at top {top_k}: {selected}')
    cutoff_gt = sorted(gt1, reverse=True)[selected - 1]
    cutoff_pred = sorted(pred1, reverse=True)[selected - 1]
#     print(f'cutoff_gt: {cutoff_gt}')
#     print(f'cutoff_pred: {cutoff_pred}')
    ind_selected_by_gt = np.where(gt1 >= cutoff_gt)[0]
    ind_selected_by_both = np.where((gt1 >= cutoff_gt) & (pred1 >= cutoff_pred))[0]
    return len(ind_selected_by_both) / len(ind_selected_by_gt)

In [18]:
top_k = 5
print(f'selected samples at top {top_k}: {get_coincidence_rate_topk(gt_import_resilience, fl_pred_import_resilience, top_k)}')

selected samples at top 5: 0.4


In [15]:
top_k = 10
print(f'selected samples at top {top_k}: {get_coincidence_rate_topk(gt_import_resilience, fl_pred_import_resilience, top_k)}')

selected samples at top 10: 0.3


In [16]:
top_k = 20
print(f'selected samples at top {top_k}: {get_coincidence_rate_topk(gt_import_resilience, fl_pred_import_resilience, top_k)}')

selected samples at top 20: 0.7


In [17]:
top_k = 30
print(f'selected samples at top {top_k}: {get_coincidence_rate_topk(gt_import_resilience, fl_pred_import_resilience, top_k)}')

selected samples at top 30: 0.9


In [19]:
# top% coincidence rate
def get_coincidence_rate(gt, pred, top_pick):
    gt1 = gt[(~np.isnan(gt)) & (~np.isnan(pred))]
    pred1 = pred[(~np.isnan(gt)) & (~np.isnan(pred))]
    total = gt1.shape[0]
    selected = int(np.ceil(top_pick * total))
#     print(f'selected samples at top pick rate {top_pick}: {selected}')
    cutoff_gt = sorted(gt1, reverse=True)[selected - 1]
    cutoff_pred = sorted(pred1, reverse=True)[selected - 1]
#     print(f'cutoff_gt: {cutoff_gt}')
#     print(f'cutoff_pred: {cutoff_pred}')
    ind_selected_by_gt = np.where(gt1 >= cutoff_gt)[0]
    ind_selected_by_both = np.where((gt1 >= cutoff_gt) & (pred1 >= cutoff_pred))[0]
    return len(ind_selected_by_both) / len(ind_selected_by_gt)

In [24]:
top_pick = 0.1
print(f'selected samples at top rate {top_pick}: {get_coincidence_rate(gt_import_resilience, fl_pred_import_resilience, top_pick)}')

selected samples at top rate 0.1: 0.5


In [25]:
top_pick = 0.2
print(f'selected samples at top rate {top_pick}: {get_coincidence_rate(gt_import_resilience, fl_pred_import_resilience, top_pick)}')

selected samples at top rate 0.2: 0.36363636363636365


In [26]:
top_pick = 0.3
print(f'selected samples at top rate {top_pick}: {get_coincidence_rate(gt_import_resilience, fl_pred_import_resilience, top_pick)}')

selected samples at top rate 0.3: 0.6875


In [27]:
top_pick = 0.5
print(f'selected samples at top rate {top_pick}: {get_coincidence_rate(gt_import_resilience, fl_pred_import_resilience, top_pick)}')

selected samples at top rate 0.5: 0.8076923076923077


In [28]:
# pearson R and spearman R
from scipy.stats import pearsonr, spearmanr

In [29]:
print("GT-PRED Pearson R: ", pearsonr(gt_import_resilience, fl_pred_import_resilience))

GT-PRED Pearson R:  PearsonRResult(statistic=0.6204021270823173, pvalue=1.1979312456057458e-06)


In [30]:
print("GT-PRED Spearman R: ", spearmanr(gt_import_resilience, fl_pred_import_resilience))

GT-PRED Spearman R:  SpearmanrResult(correlation=0.7302262443438913, pvalue=1.1940215101603559e-09)


## evaluation metrics for silo-wise centralized training results

In [31]:
west_pred = pd.read_csv('./training/noise_output_region_WEST_0.2/predict.csv', header=None)
south_pred = pd.read_csv('./training/noise_output_region_SOUTH_0.2/predict.csv', header=None)
midwest_pred = pd.read_csv('./training/noise_output_region_MIDWEST_0.2/predict.csv', header=None)
northeast_pred = pd.read_csv('./training/noise_output_region_NORTHEAST_0.2/predict.csv', header=None)
west_pred_import_resilience = west_pred.values.flatten()
south_pred_import_resilience = south_pred.values.flatten()
midwest_pred_import_resilience = midwest_pred.values.flatten()
northeast_pred_import_resilience = northeast_pred.values.flatten()

In [32]:
# MSE
print("GT-PRED WEST MSE: ", ((gt_import_resilience - west_pred_import_resilience) ** 2).mean())
print("GT-PRED SOUTH MSE: ", ((gt_import_resilience - south_pred_import_resilience) ** 2).mean())
print("GT-PRED MIDWEST MSE: ", ((gt_import_resilience - midwest_pred_import_resilience) ** 2).mean())
print("GT-PRED NORTHEAST MSE: ", ((gt_import_resilience - northeast_pred_import_resilience) ** 2).mean())

GT-PRED WEST MSE:  0.04153002378279612
GT-PRED SOUTH MSE:  0.007603086703278956
GT-PRED MIDWEST MSE:  0.005931676752481813
GT-PRED NORTHEAST MSE:  0.02984800399735259
